In [ ]:
import os
import json
from dinnovos.llms.google import GoogleLLM

## 1. Configure API Key

In [ ]:
# Configure your OpenAI API key
api_key = os.getenv("GOOGLE_API_KEY") or "your-api-key-here"
llm = GoogleLLM(
    api_key=api_key, 
    model="gemini-2.5-flash"
)

# Define tools/functions

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use"
                    }
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate",
            "description": "Perform a mathematical calculation",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "The mathematical expression to evaluate, e.g. '2 + 2'"
                    }
                },
                "required": ["expression"]
            }
        }
    }
]

# Mock function implementations

In [ ]:
def get_weather(location: str, unit: str = "celsius") -> dict:
    """Mock weather function"""
    return {
        "location": location,
        "temperature": 22 if unit == "celsius" else 72,
        "unit": unit,
        "condition": "Sunny",
        "humidity": 65
    }


def calculate(expression: str) -> dict:
    """Mock calculator function"""
    try:
        result = eval(expression)
        return {"expression": expression, "result": result}
    except Exception as e:
        return {"error": str(e)}

# Function dispatcher

In [ ]:
available_functions = {
    "get_weather": get_weather,
    "calculate": calculate
}

In [ ]:
# One call does it all!
result = llm.call_with_function_execution(
    messages=[{"role": "user", "content": "What's the weather in Bogotá?"}],
    tools=tools,
    available_functions=available_functions,
    verbose=True  # See the complete process
)

print(f"\n{'='*70}")
print("📝 FINAL RESPONSE:")
print(result['content'])
print(f"\n🔄 Iterations: {result['iterations']}")
print(f"🔧 Functions called: {len(result['function_calls'])}")

## Example 2: Multiple Functions

In [ ]:
result = llm.call_with_function_execution(
    messages=[
        {"role": "user", "content": "Compare the weather in New York and London, and calculate 15 * 8"}
    ],
    tools=tools,
    available_functions=available_functions,
    verbose=True
)

In [ ]:
print(f"\n📝 Final response: {result['content']}")
print(f"🔄 Iterations: {result['iterations']}")
print(f"🔧 Functions called: {len(result['function_calls'])}")

# Example 3: Silent mode

In [ ]:
result = llm.call_with_function_execution(
    messages=[
        {"role": "user", "content": "Calculate 100 / 4 and tell me the result"}
    ],
    tools=tools,
    available_functions=available_functions,
    verbose=False
)
    
print(f"📝 Response: {result['content']}")

# Example 4: View complete history

In [ ]:
result = llm.call_with_function_execution(
    messages=[
        {"role": "user", "content": "What's the temperature in Tokyo?"}
    ],
    tools=tools,
    available_functions=available_functions,
    temperature=0.5,
    verbose=True
)

In [ ]:
# Show all function calls
print("\n" + "=" * 70)
print("Summary of all function calls")
print("=" * 70)
for i, func_call in enumerate(result['function_calls'], 1):
    print(f"\n{i}. {func_call['name']}")
    print(f"   Args: {func_call['arguments']}")
    print(f"   Result: {func_call['result']}")